# 99 Problems in OCaml

This is taken from [99 Problems (solved) in OCaml](http://ocaml.org/learn/tutorials/99problems.html) and I will try to solve them through the days, at least 5 a day, so it will be a total of 20 days!.

## Part 1, Lists

 * ** 1. Write a function last : `'a list -> 'a option` that returns the last element of a list. (_easy_)**

In [19]:
let rec last = function
  | [x] -> Some x
  | [] -> None
  | _ :: tl -> last tl
;;

last [ "a" ; "b" ; "c" ; "d" ];;
last [];;

val last : 'a list -> 'a option = <fun>


- : string option = Some "d"


- : 'a option = None


 * ** 2. Find the last but one (last and penultimate) elements of a list. (_easy_)**

In [20]:
let rec last_two = function
  | [] | [_] -> None
  | [x;y] -> Some (x, y)
  | _ :: tl -> last_two tl
;;

last_two [ "a" ; "b" ; "c" ; "d" ];;
last_two [ "a" ];;

val last_two : 'a list -> ('a * 'a) option = <fun>


- : (string * string) option = Some ("c", "d")


- : (string * string) option = None


 * ** 3. Find the `k'`th element of a list. (_easy_)**

In [22]:
let rec at k = function
  | [] -> None
  | hd :: _ when k = 1 -> Some hd
  | _ :: tl -> at (k - 1) tl
;;

at 3 [ "a" ; "b"; "c"; "d"; "e" ];;
at 3 [ "a" ];;

val at : int -> 'a list -> 'a option = <fun>


- : string option = Some "c"


- : string option = None


 * ** 4. Find the number of elements of a list. (_easy_)**

In [24]:
let length l =
  let rec length' n = function
    | [] -> n
    | _ :: tl -> length' (n + 1) tl in
  length' 0 l
;;

length [ "a" ; "b" ; "c"];;
length [];;

val length : 'a list -> int = <fun>


- : int = 3


- : int = 0


 * ** 5. Reverse a list. (_easy_)**

In [25]:
let rev l =
  let rec rev' l' = function
    | [] -> l'
    | hd :: tl -> rev' (hd :: l') tl in
  rev' [] l
;;

rev ["a" ; "b" ; "c"];;

val rev : 'a list -> 'a list = <fun>


- : string list = ["c"; "b"; "a"]


 * **6. Find out whether a list is a palindrome. (_easy_)**

In [3]:
let is_palindrome x =
  let rev = List.rev x in
  rev = x
;;
is_palindrome [ "x" ; "a" ; "m" ; "a" ; "x" ];;
not (is_palindrome [ "a" ; "b" ]);;

val is_palindrome : 'a Core.List.t -> bool = <fun>


- : bool = true


- : bool = true


 * ** 7. Flatten a nested list structure. (_medium_)**

In [60]:
type 'a node =
  | One of 'a
  | Many of 'a node list
;;

(* This function is not tail recursive! *)
let flatten l =
  let rec flatten' acc = function
    | [] -> acc
    | One x :: tl -> flatten' (x :: acc) tl
    | Many hd :: tl -> flatten' (flatten' acc hd) tl in (* here we have a non tail call *)
  List.rev (flatten' [] l)
;;

flatten [ One "a" ; Many [ One "b" ; Many [ One "c" ; One "d" ] ; One "e" ] ];;

type 'a node = One of 'a | Many of 'a node list


val flatten : 'a node list -> 'a Core.List.t = <fun>


- : string Core.List.t = ["a"; "b"; "c"; "d"; "e"]


 * ** 8. Eliminate consecutive duplicates of list elements. (_medium_)**

In [59]:
let compress l =
  let rec aux acc = function
    | hd :: (hd' :: _ as tl) -> if hd = hd' then aux acc tl else aux (hd :: acc) tl
    | [x] -> aux (x :: acc) []
    | _ -> acc in
  List.rev (aux [] l)
;;
compress ["a";"a";"a";"a";"b";"c";"c";"a";"a";"d";"e";"e";"e";"e"];;

val compress : 'a list -> 'a Core.List.t = <fun>


- : string Core.List.t = ["a"; "b"; "c"; "a"; "d"; "e"]


 * ** 9. Pack consecutive duplicates of list elements into sublists. (_medium_)**

In [68]:
let pack l =
  let rec aux cur acc = function
    | [] -> []
    | [x] -> (x :: cur) :: acc
    | hd :: (hd' :: _ as tl) when hd = hd' -> aux (hd :: cur) acc tl
    | hd :: tl -> aux [] ((hd :: cur) :: acc) tl in
  List.rev (aux [] [] l)
;;

pack ["a";"a";"a";"a";"b";"c";"c";"a";"a";"d";"d";"e";"e";"e";"e"];;

val pack : 'a list -> 'a list Core.List.t = <fun>


- : string list Core.List.t =
[["a"; "a"; "a"; "a"]; ["b"]; ["c"; "c"]; ["a"; "a"]; ["d"; "d"];
 ["e"; "e"; "e"; "e"]]


 * ** 10. Run-length encoding of a list. (_easy_)**

In [66]:
let encode l =
  let rec aux count acc = function
    | [] -> []
    | [x] -> (count + 1, x) :: acc
    | hd :: (hd' :: _ as tl) when hd = hd' -> aux (count + 1) acc tl
    | hd :: tl -> aux 0 ((count + 1, hd) :: acc) tl in
  List.rev (aux 0 [] l)
;;

encode ["a";"a";"a";"a";"b";"c";"c";"a";"a";"d";"d";"e";"e";"e";"e"];;

val encode : 'a list -> (int * 'a) Core.List.t = <fun>


- : (int * string) Core.List.t =
[(4, "a"); (1, "b"); (2, "c"); (2, "a"); (2, "d"); (4, "e")]


 * **11. Modified run-length encoding. (_easy_)**
 
Modify the result of the previous problem in such a way that if an element has no duplicates it is simply copied into the result list. Only elements with duplicates are transferred as (N E) lists.

Since OCaml lists are homogeneous, one needs to define a type to hold both single elements and sub-lists.

In [1]:
type 'a rle =
  | One of 'a
  | Many of int * 'a
;;

type 'a rle = One of 'a | Many of int * 'a


In [18]:
let encode l =
  let rle count item = 
    if count = 1 then One item else Many (count + 1, item) in
  let rec aux count acc = function
    | [] -> []
    | [x] -> rle count x :: acc
    | hd :: (hd' :: _ as tl) when hd = hd' -> aux (count + 1) acc tl
    | hd :: tl -> aux 0 (rle count hd :: acc) tl in
  List.rev (aux 0 [] l)
;;

encode ["a";"a";"a";"a";"b";"c";"c";"a";"a";"d";"e";"e";"e";"e"];;

val encode : 'a list -> 'a rle Core.List.t = <fun>


- : string rle Core.List.t =
[Many (4, "a"); Many (1, "b"); One "c"; One "a"; Many (1, "d");
 Many (4, "e")]


 * **12. Decode a run-length encoded list. (_medium_)**

In [17]:
let decode l =
  let rec many acc count item =
    if count = 0 then acc
    else many (item :: acc) (count - 1) item in
  let rec aux acc = function
    | [] -> acc
    | (One item) :: tl -> aux (item :: acc) tl
    | (Many (count, item)) :: tl -> aux (many acc count item) tl in
  aux [] (List.rev l)
;;

decode [Many (4,"a"); One "b"; Many (2,"c"); Many (2,"a"); One "d"; Many (4,"e")];;

val decode : 'a rle Core.List.t -> 'a list = <fun>


- : string list =
["a"; "a"; "a"; "a"; "b"; "c"; "c"; "a"; "a"; "d"; "e"; "e"; "e"; "e"]


 * **13. Run-length encoding of a list (direct solution). (_medium_)**

This is basically the same as **11**, so I don't understand the question...

 * **14. Duplicate the elements of a list. (_easy_)**

In [21]:
let duplicate l =
  let rec aux acc = function
    | hd :: tl -> aux (hd :: hd :: acc) tl
    | _ -> acc in
  aux [] (List.rev l)
;;

duplicate ["a";"b";"c";"c";"d"];;

val duplicate : 'a Core.List.t -> 'a list = <fun>


- : string list = ["a"; "a"; "b"; "b"; "c"; "c"; "c"; "c"; "d"; "d"]


 * **15. Replicate the elements of a list a given number of times. (_medium_)**

In [24]:
let replicate l n =
  let rec many acc count item =
    if count = 0 then acc
    else many (item :: acc) (count - 1) item in
  let rec aux acc = function
    | hd :: tl -> aux (many acc n hd) tl
    | _ -> acc in
  aux [] (List.rev l)
;;

replicate ["a";"b";"c"] 3;;

val replicate : 'a Core.List.t -> int -> 'a list = <fun>


- : string list = ["a"; "a"; "a"; "b"; "b"; "b"; "c"; "c"; "c"]


 * **16. Drop every N'th element from a list. (_medium_)**

In [3]:
let drop l n =
  let rec drop' acc count = function
    | _ :: tl when count = n -> drop' acc 1 tl
    | hd :: tl -> drop' (hd :: acc) (count + 1) tl
    | _ -> acc in
  List.rev (drop' [] 1 l)
;;
drop ["a";"b";"c";"d";"e";"f";"g";"h";"i";"j"] 3;;

val drop : 'a list -> int -> 'a Core.List.t = <fun>


- : string Core.List.t = ["a"; "b"; "d"; "e"; "g"; "h"; "j"]


 * **17. Split a list into two parts; the length of the first part is given. (_easy_)**

In [13]:
let split l n =
  let rec aux acc count = function
    | hd :: tl when count = n -> List.rev(hd :: acc), tl
    | hd :: tl -> aux (hd :: acc) (count + 1) tl
    | [] -> List.rev(acc), [] in
  aux [] 1 l
;;
split ["a";"b";"c";"d";"e";"f";"g";"h";"i";"j"] 3 ;;
split ["a";"b";"c";"d"] 5;;

val split : 'a list -> int -> 'a Core.List.t * 'a list = <fun>


- : string Core.List.t * string list =
(["a"; "b"; "c"], ["d"; "e"; "f"; "g"; "h"; "i"; "j"])


- : string Core.List.t * string list = (["a"; "b"; "c"; "d"], [])


 * **18. Extract a slice from a list. (_medium_)**
 
 Given two indices, `i` and `k`, the slice is the list containing the elements between the `i`'th and `k`'th element of the original list (both limits included). Start counting the elements with 0 (this is the way the `List` module numbers elements).

In [26]:
let slice i k l =
  let rec aux index acc = function
    | _ :: _ when index > k -> List.rev acc
    | hd :: tl when index >= i -> aux (index + 1) (hd :: acc) tl
    | _ :: tl -> aux (index + 1) acc tl
    | _ -> List.rev acc in
  if i > k then raise (Invalid_argument "slice") else aux 0 [] l
;;

slice 2 6 ["a";"b";"c";"d";"e";"f";"g";"h";"i";"j"];;

val slice : int -> int -> 'a list -> 'a Core.List.t = <fun>


- : string Core.List.t = ["c"; "d"; "e"; "f"; "g"]


 * **19. Rotate a list N places to the left. (_medium_)**

In [31]:
let rotate n l =
  let len = List.length l in
  let pos = (n mod len + len) mod len in (* don't ask me... *)
  let rec aux index acc = function
    | hd :: tl when index = pos -> tl @ (List.rev (hd :: acc))
    | hd :: tl -> aux (index + 1) (hd :: acc) tl
    | _ -> acc in
  aux 1 [] l
;;

rotate 3 ["a"; "b"; "c"; "d"; "e"; "f"; "g"; "h"];;

val rotate : int -> 'a Core.List.t -> 'a Base__List.t = <fun>


- : string Base__List.t = ["d"; "e"; "f"; "g"; "h"; "a"; "b"; "c"]


 * **20. Remove the `K`'th element from a list. (_easy_)**

The first element of the list is numbered 0, the second 1,...

In [33]:
let remove_at k list =
  let rec aux index acc = function
    | _ :: tl when index = k -> (List.rev acc) @ tl
    | hd :: tl -> aux (index + 1) (hd :: acc) tl
    | _ -> acc in
  aux 0 [] list
;;

remove_at 1 ["a";"b";"c";"d"];;

val remove_at : int -> 'a list -> 'a Base__List.t = <fun>


- : string Base__List.t = ["a"; "c"; "d"]
